# Feature

> A collection of tools to extract features from SMILES, proteins, etc.

In [ ]:
#| default_exp feature

In [ ]:
#| hide
import sys
sys.path.append("/notebooks/katlas")
from nbdev.showdoc import *
%matplotlib inline

In [ ]:
#| export
from transformers import T5Tokenizer, T5EncoderModel, T5Model
import torch
import re
from katlas.core import Data
import seaborn as sns
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
import pandas as pd
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from sklearn.preprocessing import StandardScaler
import joblib

from fastbook import *
from fairscale.nn.data_parallel import FullyShardedDataParallel as FSDP
from fairscale.nn.wrap import enable_wrap, wrap
import esm
from tqdm.notebook import tqdm; tqdm.pandas()
import gc
from sklearn import set_config
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap.umap_ import UMAP
# from umap import UMAP
set_config(transform_output="pandas")

## Features from SMILES

### RDKit descriptors

In [ ]:
#| export
def get_rdkit(df, # df with index as ID
             col = "SMILES", # colname of smile
             normalize = True, # normalize features using StandardScaler()
            ):
    "Extract ~209 features from smiles via rdkit.Chem.Descriptors, and remove duplicate features"
    
    mols = [Chem.MolFromSmiles(smi) for smi in df[col]]
    desc_names = [desc_name[0] for desc_name in Descriptors.descList]
    desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)
    desc_values = [desc_calc.CalcDescriptors(mol) for mol in mols]
    feature_df = pd.DataFrame(np.stack(desc_values), index=df.index,columns=desc_names)
    
    if normalize:
        feature_df = StandardScaler().fit_transform(feature_df)
        
    # feature_df = feature_df.reset_index()
    return feature_df

In [ ]:
aa = Data.get_aa_info().set_index('aa')

In [ ]:
aa.head()

,Name,SMILES,MW,pKa1,pKb2,pKx3,pl4,H,VSC,P1,P2,SASA,NCISC,phospho
aa,,,,,,,,,,,,,,
A,Alanine,C[C@@H](C(=O)O)N,89.10,2.34,9.69,NaN,6.00,0.62,27.5,8.1,0.046,1.181,0.007187,0
C,Cysteine,C([C@@H](C(=O)O)N)S,121.16,1.96,10.28,8.18,5.07,0.29,44.6,5.5,0.128,1.461,-0.036610,0
D,Aspartic acid,C([C@@H](C(=O)O)N)C(=O)O,133.11,1.88,9.60,3.65,2.77,-0.90,40.0,13.0,0.105,1.587,-0.023820,0
E,Glutamic acid,C(CC(=O)O)[C@@H](C(=O)O)N,147.13,2.19,9.67,4.25,3.22,-0.74,62.0,12.3,0.151,1.862,0.006802,0
F,Phenylalanine,c1ccc(cc1)C[C@@H](C(=O)O)N,165.19,1.83,9.13,NaN,5.48,1.19,115.5,5.2,0.290,2.228,0.037552,0


In [ ]:
aa_rdkit = get_rdkit(aa, 'SMILES')

In [ ]:
aa_rdkit.head()

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,AvgIpc,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA8,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA9,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAromaticCarbocycles,NumAromaticHeterocycles,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount,MolLogP,MolMR,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_COO2,fr_C_O,fr_C_O_noCOO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_Nhpyrrole,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amide,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_barbitur,fr_benzene,fr_benzodiazepine,fr_bicyclic,fr_diazo,fr_dihydropyridine,fr_epoxide,fr_ester,fr_ether,fr_furan,fr_guanido,fr_halogen,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
aa,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A,-1.573701,-1.573701,1.193554,0.430590,-0.375462,-1.439770,-1.410267,-1.439436,-1.491575,0.0,-0.392233,0.228166,-0.308911,-0.398345,1.656688,0.103007,-0.780051,-0.526903,1.203883,-1.415519,1.465101,-1.347549,1.162418,-0.554270,0.678178,-1.660842,0.058766,-0.928528,-1.420966,-1.372879,-1.406169,-1.434358,-1.395786,-1.364454,-1.289650,-1.287461,-1.371840,-1.490293,-1.312462,-1.397762,0.661861,-0.443256,-1.488584,-1.704924,-1.164169,-1.472171,-1.088274,-0.113045,-0.213201,-0.308607,0.0,-0.521596,-0.682582,-0.458413,-0.308393,0.0,-0.647398,-0.224260,-0.480351,-0.694405,-0.68823,-0.882977,-0.213201,-0.432331,-0.495561,-0.290532,-0.695145,-0.511968,0.0,-0.308607,-0.482124,0.0,-0.308607,-0.510698,-1.373429,-0.884527,-0.45843,-0.241607,-0.504435,0.0,-0.213201,0.0,-0.945646,-0.535127,-0.661160,0.0,-0.922850,-0.621218,-0.611041,0.790873,-0.440926,-0.455591,-0.769976,-0.626017,-0.387091,-0.285797,-0.779939,-0.748232,-0.483822,0.342808,-0.393932,-0.462400,0.572190,0.387283,0.349099,-1.412520,-0.883541,-0.955395,0.0,-0.213201,-0.213201,-0.458831,-0.308607,-0.493197,-1.192079,-1.091089,-0.960437,-1.707117,0.0,-0.213201,-0.213201,-0.556890,-0.082356,-1.373751,-0.308607,-0.308607,-0.308607,0.0,0.0,-0.291386,-0.308607,-0.213201,-0.308607,-0.308607,-0.458831,-0.308607,0.0,0.0,0.0,-0.213201,-0.428746,-0.2913

### Morgan fingerprint

In [ ]:
#| export
def get_morgan(df, # a dataframe contains ID and SMILES columns
               col = "SMILES", # set smiles columne name
              ):
    "Like `smi2prop`, get 2048 morgan feature (0/1) given a dataframe that contains ID&smiles"
    mols = [Chem.MolFromSmiles(smi) for smi in df[col]]
    morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) for mol in mols]
    fp_df = pd.DataFrame(np.array(morgan_fps), index=df.index)
    fp_df.columns = "morgan_" + fp_df.columns.astype(str)
    return fp_df

In [ ]:
aa_morgan = get_morgan(aa, 'SMILES')

In [ ]:
aa_morgan.head()

,morgan_0,morgan_1,morgan_2,morgan_3,morgan_4,morgan_5,morgan_6,morgan_7,morgan_8,morgan_9,morgan_10,morgan_11,morgan_12,morgan_13,morgan_14,morgan_15,morgan_16,morgan_17,morgan_18,morgan_19,morgan_20,morgan_21,morgan_22,morgan_23,morgan_24,morgan_25,morgan_26,morgan_27,morgan_28,morgan_29,morgan_30,morgan_31,morgan_32,morgan_33,morgan_34,morgan_35,morgan_36,morgan_37,morgan_38,morgan_39,morgan_40,morgan_41,morgan_42,morgan_43,morgan_44,morgan_45,morgan_46,morgan_47,morgan_48,morgan_49,morgan_50,morgan_51,morgan_52,morgan_53,morgan_54,morgan_55,morgan_56,morgan_57,morgan_58,morgan_59,morgan_60,morgan_61,morgan_62,morgan_63,morgan_64,morgan_65,morgan_66,morgan_67,morgan_68,morgan_69,morgan_70,morgan_71,morgan_72,morgan_73,morgan_74,morgan_75,morgan_76,morgan_77,morgan_78,morgan_79,morgan_80,morgan_81,morgan_82,morgan_83,morgan_84,morgan_85,morgan_86,morgan_87,morgan_88,morgan_89,morgan_90,morgan_91,morgan_92,morgan_93,morgan_94,morgan_95,morgan_96,morgan_97,morgan_98,morgan_99,morgan_100,morgan_101,morgan_102,morgan_103,morgan_104,morgan_105,morgan_106,morgan_107,morgan_108,morgan_109,morgan_110,morgan_111,morgan_112,morgan_113,morgan_114,morgan_115,morgan_116,morgan_117,morgan_118,morgan_119,morgan_120,morgan_121,morgan_122,morgan_123,morgan_124,morgan_125,morgan_126,morgan_127,morgan_128,morgan_129,morgan_130,morgan_131,morgan_132,morgan_133,morgan_134,morgan_135,morgan_136,morgan_137,morgan_138,morgan_139,morgan_140,morgan_141,morgan_142,morgan_143,morgan_144,morgan_145,morgan_146,morgan_147,morgan_148,morgan_149,morgan_150,morgan_151,morgan_152,morgan_153,morgan_154,morgan_155,morgan_156,morgan_157,morgan_158,morgan_159,morgan_160,morgan_161,morgan_162,morgan_163,morgan_164,morgan_165,morgan_166,morgan_167,morgan_168,morgan_169,morgan_170,morgan_171,morgan_172,morgan_173,morgan_174,morgan_175,morgan_176,morgan_177,morgan_178,morgan_179,morgan_180,morgan_181,morgan_182,morgan_183,morgan_184,morgan_185,morgan_186,morgan_187,morgan_188,morgan_189,morgan_190,morgan_191,morgan_192,morgan_193,morgan_194,morgan_195,morgan_196,morgan_197,morgan_198,morgan_199,morgan_200,morgan_201,morgan_202,morgan_203,morgan_204,morgan_205,morgan_206,morgan_207,morgan_208,morgan_209,morgan_210,morgan_211,morgan_212,morgan_213,morgan_214,morgan_215,morgan_216,morgan_217,morgan_218,morgan_219,morgan_220,morgan_221,morgan_222,morgan_223,morgan_224,morgan_225,morgan_226,morgan_227,morgan_228,morgan_229,morgan_230,morgan_231,morgan_232,morgan_233,morgan_234,morgan_235,morgan_236,morgan_237,morgan_238,morgan_239,morgan_240,morgan_241,morgan_242,morgan_243,morgan_244,morgan_245,morgan_246,morgan_247,morgan_248,morgan_249,morgan_250,morgan_251,morgan_252,morgan_253,morgan_254,morgan_255,morgan_256,morgan_257,morgan_258,morgan_259,morgan_260,morgan_261,morgan_262,morgan_263,morgan_264,morgan_265,morgan_266,morgan_267,morgan_268,morgan_269,morgan_270,morgan_271,morgan_272,morgan_273,morgan_274,morgan_275,morgan_276,morgan_277,morgan_278,morgan_279,morgan_280,morgan_281,morgan_282,morgan_283,morgan_284,morgan_285,morgan_286,morgan_287,morgan_288,morgan_289,morgan_290,morgan_291,morgan_292,morgan_293,morgan_294,morgan_295,morgan_296,morgan_297,morgan_298,morgan_299,morgan_300,morgan_301,morgan_302,morgan_303,morgan_304,morgan_305,morgan_306,morgan_307,morgan_308,morgan_309,morgan_310,morgan_311,morgan_312,morgan_313,morgan_314,morgan_315,morgan_316,morgan_317,morgan_318,morgan_319,morgan_320,morgan_321,morgan_322,morgan_323,morgan_324,morgan_325,morgan_326,morgan_327,morgan_328,morgan_329,morgan_330,morgan_331,morgan_332,morgan_333,morgan_334,morgan_335,morgan_336,morgan_337,morgan_338,morgan_339,morgan_340,morgan_341,morgan_342,morgan_343,morgan_344,morgan_345,morgan_346,morgan_347,morgan_348,morgan_349,morgan_350,morgan_351,morgan_352,morgan_353,morgan_354,morgan_355,morgan_356,morgan_357,morgan_358,morgan_359,morgan_360,morgan_361,morgan_362,morgan_363,morgan_364,morgan_365,morgan_366,morgan_367,morgan_368,morgan_369,morgan_370,morgan_371,morgan_372,morgan

## Features from protein sequence

### ESM2

In [ ]:
#| export
def get_esm(df, 
                   col = 'sequence', # colname of a.a. sequence
                   model_name = "esm2_t33_650M_UR50D", # The name of the ESM model to use for the embeddings.
                  ):
    """
    Extract 1280 esmfold2 embeddings from protein sequence.
    """
    
    # Initialize distributed world with world_size 1
    if not torch.distributed.is_initialized():
        url = "tcp://localhost:23456"
        torch.distributed.init_process_group(backend="nccl", init_method=url, world_size=1, rank=0)
    
    #get number of repr layers
    match = re.search(r'_t(\d+)_', model_name)
    number = int(match.group(1))
    print(f"repr_layers number for model {model_name} is {number}.")
    print("You can also choose other esm2 models:",
          "\nesm2_t48_15B_UR50D\nesm2_t36_3B_UR50D\nesm2_t33_650M_UR50D\nesm2_t30_150M_UR50D\nesm2_t12_35M_UR50D\nesm2_t6_8M_UR50D\n")

    # Download model data from the hub
    model_data, regression_data = esm.pretrained._download_model_and_regression_data(model_name)

    # Initialize the model with FSDP wrapper
    fsdp_params = dict(
        mixed_precision=True,
        flatten_parameters=True,
        state_dict_device=torch.device("cpu"),  # reduce GPU mem usage
        cpu_offload=True,  # enable cpu offloading
    )

    with enable_wrap(wrapper_cls=FSDP, **fsdp_params):
        model, vocab = esm.pretrained.load_model_and_alphabet_core(
            model_name, model_data, regression_data
        )
        batch_converter = vocab.get_batch_converter()
        model.eval()

        # Wrap each layer in FSDP separately
        for name, child in model.named_children():
            if name == "layers":
                for layer_name, layer in child.named_children():
                    wrapped_layer = wrap(layer)
                    setattr(child, layer_name, wrapped_layer)
        model = wrap(model)

        # Define the feature extraction function
        def esm_embeddings(r, colname=col):
            data = [('protein', r[colname])]
            labels, strs, tokens = batch_converter(data)
            with torch.no_grad():
                results = model(tokens.cuda(), repr_layers=[number], return_contacts=False)
            rpr = results["representations"][number].squeeze()
            rpr = rpr[1 : len(r[colname]) + 1].mean(0).detach().cpu().numpy()

            del results, labels, strs, tokens, data #especially need to delete those on cuda: tokens, results
            gc.collect()

            return rpr
        
        # Apply the feature extraction function to each row in the DataFrame
        series = df.progress_apply(esm_embeddings, axis=1)
        df_feature = pd.DataFrame(series.tolist(), index=df.index)
        df_feature.columns = 'esm_' + df_feature.columns.astype(str)

        return df_feature

In [ ]:
# df = Data.get_kinase_info_full().set_index('kinase')
# sample = df[:5]
# esmfeature = get_esm(sample,'sequence')
# esmfeature.head()

### T5

XL-uniref50

In [ ]:
#| export
def get_t5(df, 
           col = 'sequence'
          ):

    # Reference: https://github.com/agemagician/ProtTrans/tree/master/Embedding/PyTorch/Advanced
    # Load the tokenizer
    tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)

    # Load the model
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc").to('cuda')

    # Set the model precision based on the device
    model.half()
    
    def T5_embeddings(sequence):
        seq_len = len(sequence)
        # Prepare the protein sequences as a list
        sequence = [" ".join(list(re.sub(r"[UZOB]", "X", sequence)))]

        # Tokenize sequences and pad up to the longest sequence in the batch
        ids = tokenizer.batch_encode_plus(sequence, add_special_tokens=True, padding="longest")
        input_ids = torch.tensor(ids['input_ids']).to('cuda')
        attention_mask = torch.tensor(ids['attention_mask']).to('cuda')

        # Generate embeddings
        with torch.no_grad():
            embedding_rpr = model(input_ids=input_ids, attention_mask=attention_mask)

        emb_mean = embedding_rpr.last_hidden_state[0][:seq_len].detach().cpu().numpy().mean(axis=0)

        return emb_mean

    series = df[col].progress_apply(T5_embeddings)
        

    T5_feature = pd.DataFrame(series.tolist(),index=df.index)
    T5_feature.columns = 'T5_' + T5_feature.columns.astype(str)
    
    return T5_feature

In [ ]:
# t5feature = get_t5(sample,'sequence')
# t5feature.head()

XL-BFD

In [ ]:
#| export
def get_t5_bfd(df, 
           col = 'sequence'
          ):

    # Reference: https://github.com/agemagician/ProtTrans/tree/master/Embedding/PyTorch/Advanced
    # Load the tokenizer
    tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_bfd', do_lower_case=False)

    model = T5Model.from_pretrained("Rostlab/prot_t5_xl_bfd").to('cuda')

    model.eval()
    
    def T5_embeddings_bfd(sequence, device = 'cuda'):

        seq_len = len(sequence)

        # Prepare the protein sequences as a list
        sequence = [" ".join(list(re.sub(r"[UZOB]", "X", sequence)))]

        # Tokenize sequences and pad up to the longest sequence in the batch
        ids = tokenizer.batch_encode_plus(sequence, add_special_tokens=True, padding="longest")
        input_ids = torch.tensor(ids['input_ids']).to(device)
        attention_mask = torch.tensor(ids['attention_mask']).to(device)

        # Generate embeddings
        with torch.no_grad():
            embedding_rpr = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids = input_ids)

        emb_mean = embedding_rpr.last_hidden_state[0][:seq_len].detach().cpu().numpy().mean(axis=0)

        return emb_mean

    series = df[col].progress_apply(T5_embeddings_bfd)
        

    T5_feature = pd.DataFrame(series.tolist(),index=df.index)
    T5_feature.columns = 'T5bfd_' + T5_feature.columns.astype(str)
    
    return T5_feature

In [ ]:
# t5bfd = get_t5_bfd(sample,'sequence')
# t5bfd.head()

## Dimensionality reduction

In [ ]:
#| export
def reduce_feature(df, 
               n=2, 
               method='pca',
               complexity = 20, # umap default: 15, tsne default: 30
               load=None, # load a previous model
               save=None, # pkl file to be saved, e.g. pca_model.pkl
               seed =123):
    
    "Reduce the dimensionality given a dataframe of values"
    
    if load is not None:
        reducer = joblib.load(load)
    else:
        if method == 'pca':
            reducer = PCA(n_components=n, random_state=seed)
        elif method == 'tsne':
            reducer = TSNE(n_components=n,
                           random_state=seed, 
                           perplexity = complexity # default from official is 30 
                          )
        elif method == 'umap':
            reducer = UMAP(n_components=n, 
                           random_state=seed, 
                           n_neighbors=complexity # default from official is 15, try 15-200
                          )
        else:
            raise ValueError('Invalid method specified')

    proj = reducer.fit_transform(df)
    embedding_df = pd.DataFrame(proj).set_index(df.index)
    embedding_df.columns = [f"{method.upper()}_{i}" for i in range(1, n + 1)]

    if save is not None:
        path = Path(save)
        path.parent.mkdir(exist_ok=True)
        
        joblib.dump(reducer, save)

    return embedding_df

In [ ]:
# reduced = reduce_feature(t5bfd,32, 'pca')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()